In [2]:
import pandas as pd
import os 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score

In [3]:
#finding the path of the files for the CVSs
glcm_path = r'C:\Users\User\OneDrive\Desktop\Lagos-FW-2024-Analysis\Lagos-FW-2024-Analysis-1\kasbit_glcm_results'
lbp_path = r'C:\Users\User\OneDrive\Desktop\Lagos-FW-2024-Analysis\Lagos-FW-2024-Analysis-1\kasbit_lbp_results'

In [12]:
#load and fuse the data

def load_and_fuse_data(glcm_path, lbp_path):
    combined_data = []

    for file_name in os.listdir(glcm_path):
        if file_name.endswith('_glcm_result.csv'):
            fabric = file_name.replace('_glcm_result.csv', '') #extract class names
            glcm_df = pd.read_csv(os.path.join(glcm_path, file_name)) #load glcm data
            lbp_file_path = os.path.join(lbp_path, f'{fabric}_features.csv') #load LBP data

            if os.path.exists(lbp_file_path):
                lbp_df = pd.read_csv(lbp_file_path)

                fused_data = pd.merge(glcm_df, lbp_df, on='filename')  # Merge on filename
                
                fused_data['fabric_type'] = fabric  # Add class label
                combined_data.append(fused_data)

    return pd.concat(combined_data, ignore_index=True)

In [ ]:
df = load_and_fuse_data(glcm_path, lbp_path)

X = df.drop(columns=['filename','fabric_type'])
y = df['fabric_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = svm.SVC(kernel='rbf', C=1.0, gamma='scale') #rbf is best for complex texture data
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred)*100, "%")
print(classification_report(y_test, y_pred))